# Web Scraping
1. *https://www.idefix.com/kategori/Kitap/Bilim/Populer-Bilim/grupno=00206?ShowNotForSale=True*
adresine gidin. Burada idefix.com sitesinde, Populer Bilim alanindaki kitaplar, satis rakamlarina gore siralanmistir. Bu listenin sadece ilk 10 sayfasinda yer alan kitaplari ve bu kitaplara ait bir kisim verileri derlemek istiyoruz.

2. Once bir sonraki sayfa icin verilen linkleri, bunlari kullanarak ilk 10 sayfada verilen kitap linklerini derleyin.

3. Her bir kitaba ait su bilgileri, bir data frame olarak bir araya getirin:
--kitabin adi--yazari--Ilk Baski Yili--Barkod No--satis fiyati--puanlamasi(kac kisi puan vermis ve puan ortalamasi seklinde)--idefix.com sayfasinda verilen linki--derledigimiz ilk 10 sayfada yer alan diger kitaplar arasindaki satis siralamasi

In [1]:
from fake_useragent import UserAgent
import requests

header = {'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36"}

from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [2]:
base_site = "https://www.idefix.com/kategori/Kitap/Bilim/Populer-Bilim/grupno=00206?ShowNotForSale=True"
base_site

'https://www.idefix.com/kategori/Kitap/Bilim/Populer-Bilim/grupno=00206?ShowNotForSale=True'

In [3]:
response = requests.get(base_site, headers=header)
html = response.content
response.status_code

200

In [4]:
soup = BeautifulSoup(html, 'lxml')
a_links= soup.find_all("a", {"class": "product-image"})
len(a_links)

36

In [5]:
a_links[35].get("href")

'/Kitap/Hiperuzay/Michio-Kaku/Bilim/Populer-Bilim/urunno=0001936902001'

In [6]:
note_urls = [urljoin(base_site, l.get('href')) for l in a_links]
note_urls[:5]

['https://www.idefix.com/Kitap/Dura-Mater/Serkan-Karaismailoglu/Bilim/Populer-Bilim/urunno=0001934377001',
 'https://www.idefix.com/Kitap/Zamanin-Kisa-Tarihi/Bilim/Populer-Bilim/urunno=0000000562120',
 'https://www.idefix.com/Kitap/Doganin-Isaretlerini-Okumanin-Kaybolmus-Sanati/Bilim/Populer-Bilim/urunno=0001785184001',
 'https://www.idefix.com/Kitap/IFA-Insanin-Fabrika-Ayarlari-3Kitap-Sinirlari-Asmak/Bilim/Populer-Bilim/urunno=0001895030001',
 'https://www.idefix.com/Kitap/IFA-Insanin-Fabrika-Ayarlari-1Kitap-Beden/Sinan-Canan/Bilim/Populer-Bilim/urunno=0001843838001']

In [7]:
links = []

for i in range(1,11):
    if i == 1:
        base_site = "https://www.idefix.com/kategori/Kitap/Bilim/Populer-Bilim/grupno=00206?ShowNotForSale=True"
    else:        
        base_site = "https://www.idefix.com/kategori/Kitap/Bilim/Populer-Bilim/grupno=00206?ShowNotForSale=True&Page="+str(i)
    response = requests.get(base_site)
    html = response.content
    soup = BeautifulSoup(html, "lxml")

    a_links= soup.find_all("a", {"class": "product-image"})
    note_urls = [urljoin(base_site, l.get('href')) for l in a_links]
    for k in range(len(note_urls)):
        links.append(note_urls[k])

In [8]:
len(links)

360

In [9]:
links;

In [10]:
links[8]

'https://www.idefix.com/Kitap/Gizlenen-Ufo-Temaslari-Ufolarla-Yasanan-Karsilasmalar-ve-ortbas-Edilen-Gercekler/Bilim/Populer-Bilim/urunno=0001947445001'

In [11]:
df = pd.DataFrame()

for i in links:
    response = requests.get(i)
    html = response.content
    soup = BeautifulSoup(html, "lxml")
    
    try:
        kitap_adi = soup.find("div", class_ = "product-info-list").find(text="Kitap Adı:").findNext("a").text
    except Exception:
        kitap_adi = "NAN"           

    try:
        yazar_adi = soup.find("div", class_ = "product-info-list").find(text="Yazar: ").findNext("a").text.strip()              
    except Exception:
        yazar_adi = "NAN"
                
    try:
        yayin_evi = soup.find("div" , class_ = "publisher").text.split(":")[1].strip()
    except Exception:
        yayin_evi = "NAN"
        
    try:
        ilk_baski_yili = soup.find("div", class_ = "product-info-list").find(text="İlk Baskı Yılı:").findNext("a").text
    except Exception:
        ilk_baski_yili = "NAN"           

    try:
        barkod = soup.find("div", class_ = "product-info-list").find(text="Barkod:").findNext("a").text
    except Exception:
        barkod = "NAN"
                
    try:
        satis_fiyati = soup.find("div", {"class" : "current-price"}).text.strip(" TL")
    except Exception:
        satis_fiyati = "NAN"
        
    try:
        puan_ortalamasi = soup.find("div", {"class" : "rate"}).findNext("span").text.split()[0]
    except Exception:
        puan_ortalamasi = "NAN"
                
    try:
        puan_veren_kisi = soup.find("div", {"class" : "rate"}).findNext("span").text.split()[2]
    except Exception:
        puan_veren_kisi = "NAN"
        
    df_link = pd.DataFrame([kitap_adi, yazar_adi, yayin_evi, ilk_baski_yili, barkod, satis_fiyati, 
                            puan_ortalamasi, puan_veren_kisi], 
                           index = ["kitap_adi", "yazar_adi", "yayin_evi", "ilk_baski_yili", "barkod", 
                                    "satis_fiyati", "puan_ortalamasi", "puan_veren_kisi"]).T
    df = pd.concat((df,df_link), axis = 0)
df

,kitap_adi,yazar_adi,yayin_evi,ilk_baski_yili,barkod,satis_fiyati,puan_ortalamasi,puan_veren_kisi
0,Dura Mater,Serkan Karaismailoğlu,Elma Yayınevi,2021,9786257112376,"42,25",3/5,17
0,Zamanın Kısa Tarihi,Stephen Hawking,Alfa Yayıncılık,2017,9786051067582,"20,8",4/5,151
0,Doğanın İşaretlerini Okumanın Kaybolmuş Sanatı,Tristan Gooley,Destek Yayınları,2018,9786053115076,"26,4",4/5,44
0,İFA: İnsanın Fabrika Ayarları 3.Kitap - Sınırl...,Sinan Canan,Tuti Kitap,2020,9786059218733,"24,0","3,5/5",24
0,İFA:İnsanın Fabrika Ayarları 1.Kitap-Beden,Sinan Canan,Tuti Kitap,2019,9786059218603,"24,0","3,5/5",49
...,...,...,...,...,...,...,...,...
0,Sonsuz Yaşamın Sırrı ve Bilim Üzerine İlgi Çek...,Alok Jha,İthaki Yayınları,2014,9786053753889,"25,2",0/5,0
0,Depremler Doğal Afet Midir? - Depremlerle Baş ...,Süleyman Pampal,Efil Yayınevi Yayınları,2010,9786054160273,"48,0",0/5,0
0,Dünya Beni Bekler Mi?,Gül Sevin Pekmezci,Kent Kitap,2011,9789944915342,"23,1",0/5,0
0,Tanrı Beyni - Beyin Neden İnanç Üretir?,"Lionel Tiger ,",Alfa Yayıncılık,2011,9786051063713,"18,2","3,5/5",9


In [27]:
df = df.reset_index(drop=True)

In [28]:
df.shape

(360, 8)

In [29]:
# ilk 10 sayfadaki 360 adet kitaba ait verileri çekerek DataFrame olarak düzenlemiş olduk.

In [30]:
# en yüksek puana ve akabinde en çok puan veren kişi sayısına göre sıraladık.

df.sort_values(by = ["puan_ortalamasi", "puan_veren_kisi"], ascending = False)

,kitap_adi,yazar_adi,yayin_evi,ilk_baski_yili,barkod,satis_fiyati,puan_ortalamasi,puan_veren_kisi
21,Karanlık Bir Dünyada Bilimin Mum Isıgı,Carl Sagan,Say Yayınları,2020,9786050207743,"45,5",5/5,6
37,Hayvanların Ne Kadar Zeki Olduğunu Anlayacak K...,Frans De Waal,Metis Yayıncılık,2017,9786053161073,"33,5",5/5,4
253,Evrenin Karanlığında Evrimin Işığı,NAN,Ginko Bilim,NAN,9786058111110,"49,0",5/5,4
32,Bilim Nasıl Çalışır?,Kolektif,Alfa Yayıncılık,2019,9786051068688,"90,0",5/5,2
66,Hayal Gücü,Jonah Lehrer,Boğaziçi Üniversitesi Yayınevi,2016,9786054787739,"19,25",5/5,2
...,...,...,...,...,...,...,...,...
355,Sonsuz Yaşamın Sırrı ve Bilim Üzerine İlgi Çek...,Alok Jha,İthaki Yayınları,2014,9786053753889,"25,2",0/5,0
356,Depremler Doğal Afet Midir? - Depremlerle Baş ...,Süleyman Pampal,Efil Yayınevi Yayınları,2010,9786054160273,"48,0",0/5,0
357,Dünya Beni Bekler Mi?,Gül Sevin Pekmezci,Kent Kitap,2011,9789944915342,"23,1",0/5,0
359,Çevreci Bilim Kitapları (6 Kitap Takım),Vijita Mukherjee,Damla Yayınevi,2013,9786053830849,"25,2",0/5,0
